In [1]:
def use_model(path_to_data: str, model, threshold: float=0.31191915, path_to_json: str='process_results.json'):
    """
    Запускает модель, переданную в качестве параметров, применяя к ней метод .predict()
    и сохраняет результат в файл process_results.json
     
    :param path_to_data: Путь к папке с данными 
    :param model: Модель, с возможностью применить метод .predict()
    :param threshold: Трешхолд
    :param path_to_json: Путь с именем файла, в котором сохраняться результаты
    """
    %tensorflow_version 1.x
    import tensorflow as tf
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from pathlib import Path
    import pandas as pd
    import json
    ! pip install keras_vggface
    from keras_vggface import utils

    base_dir = Path(path_to_data)

    filenames = list(map(lambda x: str(x.name), base_dir.glob('*.jpg')))
    
    df = pd.DataFrame({'file_name': filenames})

    def preprocess_input_facenet(image):
        """
        image -- тензор размера (1, H, W, 3)
        return: картинка, с примененным preprocess_input(..., version=2) из keras_vggface 
        """
   

        preprocessed = utils.preprocess_input(image, version=2)

        return preprocessed

    image_gen = ImageDataGenerator(preprocessing_function=preprocess_input_facenet)

    data_gen = image_gen.flow_from_dataframe(
                                dataframe=df,
                                directory=base_dir,
                                x_col='file_name',
                                weight_col=None,
                                target_size=(224, 224),
                                color_mode="rgb",
                                classes=None,
                                class_mode=None,
                                shuffle=True,
                                seed=None,
                                save_to_dir=None,
                                save_prefix="",
                                subset=None,
                                interpolation="nearest",
                                validate_filenames=True)

    preds = model.predict(data_gen)

    df['preds'] = preds
    df['preds'] = df['preds'].apply(lambda x: x>threshold).map({True: 'male', False: 'female'}) 

    json_dict = json.dumps(dict(zip(df['file_name'], df['preds'])))

    with open(path_to_json, 'w') as f:
        json.dump(json_dict, f)